In [95]:
import requests
import json
import pandas as pd
import time
import progressbar

In [108]:
def get_advanced_data(matricula):

    cookies = {
        'altocontraste': 'off',
        'fontePersonalizada': '14',
    }

    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:70.0) Gecko/20100101 Firefox/70.0',
        'Accept': 'application/json, text/javascript, */*; q=0.01',
        'Accept-Language': 'en-US,en;q=0.5',
        'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8',
        'X-Requested-With': 'XMLHttpRequest',
        'Origin': 'https://acessoainformacao.aruana.go.gov.br',
        'DNT': '1',
        'Connection': 'keep-alive',
        'Referer': 'https://acessoainformacao.aruana.go.gov.br/cidadao/transparencia/mgservidores',
    }

    data = {
      'multi_request': 'true',
      'params': '{"0-5dsuo6":{"ano":2019,"mes":"9","matricula":%s,"acao":"megasoft/servidoresDetalhe"}}' %matricula
    }

    response = requests.post('https://acessoainformacao.aruana.go.gov.br/api', 
                             headers=headers, cookies=cookies, data=data)
    
    df = pd.DataFrame.from_dict(response.json(),orient='index')
    return df

In [177]:
def get_payments_go(matricula):


    cookies = {
        'altocontraste': 'off',
        'fontePersonalizada': '14',
    }

    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:70.0) Gecko/20100101 Firefox/70.0',
        'Accept': 'application/json, text/javascript, */*; q=0.01',
        'Accept-Language': 'en-US,en;q=0.5',
        'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8',
        'X-Requested-With': 'XMLHttpRequest',
        'Origin': 'https://acessoainformacao.aruana.go.gov.br',
        'DNT': '1',
        'Connection': 'keep-alive',
        'Referer': 'https://acessoainformacao.aruana.go.gov.br/cidadao/transparencia/mgservidores',
    }

    data = {
      'multi_request': 'true',
      'params': '{"0-cojg5h":{"ano":2019,"mes":"10","matricula":%s,"acao":"megasoft/servidoresDetalhe"}}' %matricula
    }

    response = requests.post('https://acessoainformacao.aruana.go.gov.br/api', 
                             headers=headers, cookies=cookies, data=data)
    
    df = pd.DataFrame.from_dict(response.json(),orient='index')
    return df

## Matriculation numbers from october

In [35]:
filepath = '../data/folha_pgto_aruana_oct.csv'
df = pd.read_csv(filepath, encoding='latin-1',sep=';')
df.columns = ['matricula','nome','cargo','situacao','mes','ano']

In [36]:
df.head()

,matricula,nome,cargo,situacao,mes,ano
0,2902,ACACIO AMELIO DA ROCHA NETO,ASSISTENTE BASICO,Ativo,2019.0,Outubro
1,3050,ACARY FELIX DA SILVA FILHO,ASSISTENTE ESPECIAL 68,Ativo,2019.0,Outubro
2,2396,ADAO ALVES COSTA DE OLIVEIRA,DIRETOR ADJUNTO,Ativo,2019.0,Outubro
3,2507,ADAO GOMES PINHEIRO,ASSISTENTE ESPECIAL 68,Ativo,2019.0,Outubro
4,1022,ADAO MARTINS ARAGAO,PENSIONISTA,Pensionista,2019.0,Outubro


In [37]:
matriculas = df['matricula'].unique().tolist()

In [41]:
print (len(matriculas))
print (matriculas[0])

520
2902


In [114]:
list2=[]

with progressbar.ProgressBar(max_value=550) as bar:
    for i,matricula in enumerate(matriculas):
        
        advanced_stats = get_advanced_data(matricula)
        list2.append(advanced_stats)
        bar.update(i)

100% (550 of 550) |######################| Elapsed Time: 0:10:52 Time:  0:10:52


In [115]:
len(list2)

520

In [121]:
advanced_info = pd.concat(list2,ignore_index=True,sort=False)

In [153]:
advanced_info.to_excel('detalhes_servidores_go.xlsx')

In [126]:
advanced_info['tipoAdmissao'].value_counts()

Concursado                                          251
Comissionado                                        248
Tempo determinado/Processo seletivo simplificado      6
Aposentado                                            5
Pensionista (Vitalício)                               3
Pensionista Mercê                                     2
Prefeito                                              1
Vice-Prefeito                                         1
Pensionista (Temporário)                              1
Secretário                                            1
Name: tipoAdmissao, dtype: int64

In [136]:
num_comissionados = len(advanced_info[advanced_info['tipoAdmissao'] == 'Comissionado'])
num_total_servidores = len(advanced_info)
print ('pctg. comissionados: {:.2f}%'.format(100*num_comissionados/num_total_servidores))

pctg. comissionados: 47.78%


## Name matching with mayor

In [150]:
surname_prefeito = 'CARVALHO'
surnames_vice = ['ANTUNES','BARBOSA'] 

In [151]:
comissionados=advanced_info[advanced_info['tipoAdmissao']=='Comissionado']

In [152]:
comissionados[comissionados['nome'].str.contains('CARVALHO')]

,matricula,nome,cargo,situacao,dataAdmissao,mes,ano,departamento,orgao,decreto,...,movimentoComplementar,somatorioMovimentoComplementar,movimentoSessoesExtraordinarias,somatorioMovimentoSessoesExtraordinarias,movimentoRescisao,somatorioMovimentoRescisao,mesAno,chave,classificacao,observacao
72,2662,CARLOS ALBERTO CARVALHO DA SILVA,ASSISTENTE ESPECIAL 68,Ativo,12/06/2018,9,2019,SECRETARIA MUNICIPAL DE OBRAS,PREFEITURA MUNICIPAL DE ARUANA,70/2018,...,"{'total': 0, 'registros': [], 'paginaAtual': 1}",{},"{'total': 0, 'registros': [], 'paginaAtual': 1}",{},"{'total': 0, 'registros': [], 'paginaAtual': 1}",{},9/2019,"{'pagina': 1, 'tamanhoDaPagina': 5, 'matricula...",NaN,NaN
151,2984,EZER NATA CARVALHO FRANCA,ASSISTENTE ESPECIAL 68,Ativo,11/06/2019,9,2019,SECRETARIA MUNICIPAL DE OBRAS,PREFEITURA MUNICIPAL DE ARUANA,057/2019,...,"{'total': 0, 'registros': [], 'paginaAtual': 1}",{},"{'total': 0, 'registros': [], 'paginaAtual': 1}",{},"{'total': 0, 'registros': [], 'paginaAtual': 1}",{},9/2019,"{'pagina': 1, 'tamanhoDaPagina': 5, 'matricula...",NaN,NaN


In [144]:
advanced_info[advanced_info['nome'].str.contains('CARVALHO')]

,matricula,nome,cargo,situacao,dataAdmissao,mes,ano,departamento,orgao,decreto,...,movimentoComplementar,somatorioMovimentoComplementar,movimentoSessoesExtraordinarias,somatorioMovimentoSessoesExtraordinarias,movimentoRescisao,somatorioMovimentoRescisao,mesAno,chave,classificacao,observacao
31,2128,ANA EDIELLEN CARVALHO FRANÇA,PROFESSOR AN 2,Ativo,18/01/2016,9,2019,FUNDEB - (60% - EFETIVOS),PREFEITURA MUNICIPAL DE ARUANA FUNDEB,01145/2016,...,"{'total': 0, 'registros': [], 'paginaAtual': 1}",{},"{'total': 0, 'registros': [], 'paginaAtual': 1}",{},"{'total': 0, 'registros': [], 'paginaAtual': 1}",{},9/2019,"{'pagina': 1, 'tamanhoDaPagina': 5, 'matricula...",13,NaN
72,2662,CARLOS ALBERTO CARVALHO DA SILVA,ASSISTENTE ESPECIAL 68,Ativo,12/06/2018,9,2019,SECRETARIA MUNICIPAL DE OBRAS,PREFEITURA MUNICIPAL DE ARUANA,70/2018,...,"{'total': 0, 'registros': [], 'paginaAtual': 1}",{},"{'total': 0, 'registros': [], 'paginaAtual': 1}",{},"{'total': 0, 'registros': [], 'paginaAtual': 1}",{},9/2019,"{'pagina': 1, 'tamanhoDaPagina': 5, 'matricula...",NaN,NaN
151,2984,EZER NATA CARVALHO FRANCA,ASSISTENTE ESPECIAL 68,Ativo,11/06/2019,9,2019,SECRETARIA MUNICIPAL DE OBRAS,PREFEITURA MUNICIPAL DE ARUANA,057/2019,...,"{'total': 0, 'registros': [], 'paginaAtual': 1}",{},"{'total': 0, 'registros': [], 'paginaAtual': 1}",{},"{'total': 0, 'registros': [], 'paginaAtual': 1}",{},9/2019,"{'pagina': 1, 'tamanhoDaPagina': 5, 'matricula...",NaN,NaN
183,2381,HERMANO DE CARVALHO,PREFEITO MUNICIPAL DE ARUANA,Ativo,01/01/2017,9,2019,GABINETE DO PREFEITO,PREFEITURA MUNICIPAL DE ARUANA,NaN,...,"{'total': 0, 'registros': [], 'paginaAtual': 1}",{},"{'total': 0, 'registros': [], 'paginaAtual': 1}",{},"{'total': 0, 'registros': [], 'paginaAtual': 1}",{},9/2019,"{'pagina': 1, 'tamanhoDaPagina': 5, 'matricula...",NaN,NaN
330,2131,MARIA CELIA DE CARVALHO,PROFESSOR AN 2,Ativo,18/01/2016,9,2019,FUNDEB - (60% - EFETIVOS),PREFEITURA MUNICIPAL DE ARUANA FUNDEB,01145/2016,...,"{'total': 0, 'registros': [], 'paginaAtual': 1}",{},"{'total': 0, 'registros': [], 'paginaAtual': 1}",{},"{'total': 0, 'registros': [], 'paginaAtual': 1}",{},9/2019,"{'pagina': 1, 'tamanhoDaPagina': 5, 'matricula...",16,NaN
331,3039,MARIA CELIA DE CARVALHO,PROFESSOR AN 2,Ativo,19/09/2019,9,2019,AUX. DOENÇA,PREFEITURA MUNIC DE ARUANÃ FUMPREV,01145/2019,...,"{'total': 0, 'registros': [], 'paginaAtual': 1}",{},"{'total': 0, 'registros': [], 'paginaAtual': 1}",{},"{'total': 0, 'registros': [], 'paginaAtual': 1}",{},9/2019,"{'pagina': 1, 'tamanhoDaPagina': 5, 'matricula...",16,NaN
353,324,MARINEIDE MARIA DE CARVALHO,PROFESSOR DN-3,Ativo,04/04/2002,9,2019,FUNDEB - (60% - EFETIVOS),PREFEITURA MUNICIPAL DE ARUANA FUNDEB,0258/2002,...,"{'total': 0, 'registros': [], 'paginaAtual': 1}",{},"{'total': 0, 'registros': [], 'paginaAtual': 1}",{},"{'total': 0, 'registros': [], 'paginaAtual': 1}",{},9/2019,"{'pagina': 1, 'tamanhoDaPagina': 5, 'matricula...",18,NaN
391,2963,OSVALDO LEITE CARVALHO,MOTORISTA DE VEICULOS PESADOS 17,Ativo,14/05/2019,9,2019,SECRETARIA MUNICIPAL DE OBRAS,PREFEITURA MUNICIPAL DE ARUANA,047/2019,...,"{'total': 0, 'registros': [], 'paginaAtual': 1}",{},"{'total': 0, 'registros': [], 'paginaAtual': 1}",{},"{'total': 0, 'registros': [], 'paginaAtual': 1}",{},9/2019,"{'pagina': 1, 'tamanhoDaPagina': 5, 'matricula...",1,NaN


# Folha de pagamento - out

In [155]:
filepath = '../data/RelatórioDeFolhaPagamento_aruana_out.csv'
df = pd.read_csv(filepath, encoding='latin-1',sep=';')
df.columns = ['matricula','nome','cargo','situacao','mes','ano']

In [159]:
matriculas2 = df['matricula'].unique().tolist()
print (len(matriculas2))

520


In [182]:
list2=[]

with progressbar.ProgressBar(max_value=550) as bar:
    for i,matricula in enumerate(matriculas2):
        
        advanced_stats = get_payments_go(matricula)
        list2.append(advanced_stats)
        bar.update(i)

100% (550 of 550) |######################| Elapsed Time: 0:13:56 Time:  0:13:56


In [216]:
advanced_info_payments_go = pd.concat(list2,ignore_index=True,sort=False)

In [217]:
#advanced_info_payments_go.columns
advanced_info_payments_go.head()

,matricula,nome,cargo,situacao,dataAdmissao,mes,ano,departamento,orgao,decreto,...,movimentoComplementar,somatorioMovimentoComplementar,movimentoSessoesExtraordinarias,somatorioMovimentoSessoesExtraordinarias,movimentoRescisao,somatorioMovimentoRescisao,mesAno,chave,classificacao,observacao
0,2902,ACACIO AMELIO DA ROCHA NETO,ASSISTENTE BASICO,Ativo,01/02/2019,10,2019,SECRETARIA MUNICIPAL DE EDUCAÇÃO,PREFEITURA MUNICIPAL DE ARUANA,20/2019,...,"{'total': 0, 'registros': [], 'paginaAtual': 1}",{},"{'total': 0, 'registros': [], 'paginaAtual': 1}",{},"{'total': 0, 'registros': [], 'paginaAtual': 1}",{},10/2019,"{'pagina': 1, 'tamanhoDaPagina': 5, 'matricula...",NaN,NaN
1,3050,ACARY FELIX DA SILVA FILHO,ASSISTENTE ESPECIAL 68,Ativo,01/10/2019,10,2019,SECRETARIA MUNICIPAL DE OBRAS,PREFEITURA MUNICIPAL DE ARUANA,101/2019,...,"{'total': 0, 'registros': [], 'paginaAtual': 1}",{},"{'total': 0, 'registros': [], 'paginaAtual': 1}",{},"{'total': 0, 'registros': [], 'paginaAtual': 1}",{},10/2019,"{'pagina': 1, 'tamanhoDaPagina': 5, 'matricula...",NaN,NaN
2,2396,ADAO ALVES COSTA DE OLIVEIRA,DIRETOR ADJUNTO,Ativo,01/02/2017,10,2019,SECRETARIA MUNICIPAL DE EDUCAÇÃO,PREFEITURA MUNICIPAL DE ARUANA,067/2017,...,"{'total': 0, 'registros': [], 'paginaAtual': 1}",{},"{'total': 0, 'registros': [], 'paginaAtual': 1}",{},"{'total': 0, 'registros': [], 'paginaAtual': 1}",{},10/2019,"{'pagina': 1, 'tamanhoDaPagina': 5, 'matricula...",NaN,NaN
3,2507,ADAO GOMES PINHEIRO,ASSISTENTE ESPECIAL 68,Ativo,02/01/2018,10,2019,SECRETARIA MUNICIPAL DE OBRAS,PREFEITURA MUNICIPAL DE ARUANA,002/2018,...,"{'total': 0, 'registros': [], 'paginaAtual': 1}",{},"{'total': 0, 'registros': [], 'paginaAtual': 1}",{},"{'total': 0, 'registros': [], 'paginaAtual': 1}",{},10/2019,"{'pagina': 1, 'tamanhoDaPagina': 5, 'matricula...",NaN,NaN
4,1022,ADAO MARTINS ARAGAO,PENSIONISTA,Pensionista,09/12/2010,10,2019,PENSIONISTA,PREFEITURA MUNIC DE ARUANÃ FUMPREV,0039/2011,...,"{'total': 0, 'registros': [], 'paginaAtual': 1}",{},"{'total': 0, 'registros': [], 'paginaAtual': 1}",{},"{'total': 0, 'registros': [], 'paginaAtual': 1}",{},10/2019,"{'pagina': 1, 'tamanhoDaPagina': 5, 'matricula...",13,NaN


In [218]:
#print (advanced_info_payments_go.iloc[1]['somatorioMovimentoVencimentos']['somatorioProventos'])
len(advanced_info_payments_go.iloc[1]['somatorioMovimentoRescisao'])
#print (advanced_info_payments_go.iloc[1]['somatorioMovimentoRescisao']['somatorioProventos'])
#advanced_info_payments_go['movimentoVencimentos'].iloc[12]['registros'][0]['proventos']
#advanced_info_payments_go['movimentoVencimentos'].iloc[0]['registros'][0]['descontos']
def return_values_remuneracoes(x, index):
    provento_vencimento = None if len(x['somatorioMovimentoVencimentos'])==0 else x['somatorioMovimentoVencimentos']['somatorioProventos']
    provento_desconto = None if len(x['somatorioMovimentoVencimentos'])==0 else x['somatorioMovimentoVencimentos']['somatorioDescontos']
    rescisao_vencimento = None if len(x['somatorioMovimentoRescisao'])==0 else x['somatorioMovimentoRescisao']['somatorioProventos']
    rescisao_desconto = None if len(x['somatorioMovimentoRescisao'])==0 else x['somatorioMovimentoRescisao']['somatorioDescontos']
    matrix = (provento_vencimento,provento_desconto, rescisao_vencimento, rescisao_desconto)
    return matrix[index]

In [219]:
advanced_info_payments_go['outras_remuneracoes_proventos'] = advanced_info_payments_go.apply(lambda x: \
return_values_remuneracoes(x,0),axis=1)
advanced_info_payments_go['outras_remuneracoes_descontos'] = advanced_info_payments_go.apply(lambda x: \
return_values_remuneracoes(x,1),axis=1)
advanced_info_payments_go['rescisao_proventos'] = advanced_info_payments_go.apply(lambda x: \
return_values_remuneracoes(x,2),axis=1)
advanced_info_payments_go['rescisaooutras__descontos'] = advanced_info_payments_go.apply(lambda x: \
return_values_remuneracoes(x,3),axis=1)

In [220]:
advanced_info_payments_go.head()

,matricula,nome,cargo,situacao,dataAdmissao,mes,ano,departamento,orgao,decreto,...,movimentoRescisao,somatorioMovimentoRescisao,mesAno,chave,classificacao,observacao,outras_remuneracoes_proventos,outras_remuneracoes_descontos,rescisao_proventos,rescisaooutras__descontos
0,2902,ACACIO AMELIO DA ROCHA NETO,ASSISTENTE BASICO,Ativo,01/02/2019,10,2019,SECRETARIA MUNICIPAL DE EDUCAÇÃO,PREFEITURA MUNICIPAL DE ARUANA,20/2019,...,"{'total': 0, 'registros': [], 'paginaAtual': 1}",{},10/2019,"{'pagina': 1, 'tamanhoDaPagina': 5, 'matricula...",NaN,NaN,1009.72,80.77,NaN,NaN
1,3050,ACARY FELIX DA SILVA FILHO,ASSISTENTE ESPECIAL 68,Ativo,01/10/2019,10,2019,SECRETARIA MUNICIPAL DE OBRAS,PREFEITURA MUNICIPAL DE ARUANA,101/2019,...,"{'total': 0, 'registros': [], 'paginaAtual': 1}",{},10/2019,"{'pagina': 1, 'tamanhoDaPagina': 5, 'matricula...",NaN,NaN,1009.72,80.77,NaN,NaN
2,2396,ADAO ALVES COSTA DE OLIVEIRA,DIRETOR ADJUNTO,Ativo,01/02/2017,10,2019,SECRETARIA MUNICIPAL DE EDUCAÇÃO,PREFEITURA MUNICIPAL DE ARUANA,067/2017,...,"{'total': 0, 'registros': [], 'paginaAtual': 1}",{},10/2019,"{'pagina': 1, 'tamanhoDaPagina': 5, 'matricula...",NaN,NaN,1787.15,160.84,NaN,NaN
3,2507,ADAO GOMES PINHEIRO,ASSISTENTE ESPECIAL 68,Ativo,02/01/2018,10,2019,SECRETARIA MUNICIPAL DE OBRAS,PREFEITURA MUNICIPAL DE ARUANA,002/2018,...,"{'total': 0, 'registros': [], 'paginaAtual': 1}",{},10/2019,"{'pagina': 1, 'tamanhoDaPagina': 5, 'matricula...",NaN,NaN,1620.13,113.45,NaN,NaN
4,1022,ADAO MARTINS ARAGAO,PENSIONISTA,Pensionista,09/12/2010,10,2019,PENSIONISTA,PREFEITURA MUNIC DE ARUANÃ FUMPREV,0039/2011,...,"{'total': 0, 'registros': [], 'paginaAtual': 1}",{},10/2019,"{'pagina': 1, 'tamanhoDaPagina': 5, 'matricula...",13,NaN,1155.60,335.00,NaN,NaN


In [221]:
advanced_info_payments_go.to_csv('pagamentos_aruana_out.csv')